In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder

from sklearn.impute import SimpleImputer

In [27]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_seq_items', None)

In [5]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/test.csv')

In [6]:
holidays_events = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/holidays_events.csv')
oil = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/oil.csv')
stores = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/stores.csv')
trandsactions = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/transactions.csv')

In [7]:
train.set_index('id', inplace=True)
test.set_index('id', inplace=True)

In [8]:
all_data = pd.concat([train, test], sort=False)

In [247]:
all_data.head()

,date,store_nbr,family,sales,onpromotion
id,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0
1,2013-01-01,1,BABY CARE,0.0,0.0
2,2013-01-01,1,BEAUTY,0.0,0.0
3,2013-01-01,1,BEVERAGES,0.0,0.0
4,2013-01-01,1,BOOKS,0.0,0.0


In [9]:
# distinguish holiday and it takes around 1min

all_data['isHoliday'] = False

for i in range(0, len(holidays_events)):
  all_data.loc[all_data['date'] == holidays_events['date'][i], 'isHoliday'] = True

In [249]:
# There is no NA data

na_check = all_data.isna().sum() / len(all_data)
na_check = na_check[na_check >= 0.0]
na_check

date           0.000000
store_nbr      0.000000
family         0.000000
sales          0.009412
onpromotion    0.000000
isHoliday      0.000000
dtype: float64

In [250]:
oil['isHoliday'] = False

for i in range(0, len(holidays_events)):
  oil.loc[oil['date'] == holidays_events['date'][i], 'isHoliday'] = True

na_check = oil.isna().sum() / len(oil)
na_check

date          0.0
dcoilwtico    0.0
isHoliday     0.0
dtype: float64

In [10]:
oil['dcoilwtico'][0] = oil['dcoilwtico'][1]

for i in range(0, len(oil)):
  if pd.isna(oil['dcoilwtico'][i]) and i > 0:
    oil['dcoilwtico'][i] = oil['dcoilwtico'][i-1]

na_check = oil.isna().sum() / len(oil)
na_check

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


date          0.0
dcoilwtico    0.0
dtype: float64

In [11]:
# it takes around 4mins

all_data['oil_price'] = 0

for i in range(0, len(oil)):
  dt = oil['date'][i]
  price = oil['dcoilwtico'][i]

  all_data.loc[all_data['date'] == dt, 'oil_price'] = price


In [13]:
array = []

array = pd.DataFrame(array)

col1 = all_data['date'].unique()
col2 = all_data['store_nbr'].unique()

for i in col2:
  for j in col1:
    col_n = j+' '+(str)(i)
    array[col_n] = 0
  print(i)


1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28


KeyboardInterrupt: ignored

In [1]:
all_data['transactions'] = None

print(len(all_data))
for idx, row in all_data.iterrows():
  dt = row['date']
  nbr = row['store_nbr']

  col_n = dt+'_'+(str)(nbr)

  try:
    tst = pd.to_numeric(array[col_n])
    row['transactions'] = tst
  except:
    tst = None



  if idx%100000 == 0:
    print(idx)

NameError: ignored

In [14]:
# separate object data and non-object data

data_object = all_data.select_dtypes(include=object)
data_non_object = all_data.select_dtypes(exclude=object)

print(data_object.columns)
print(data_non_object.columns)

Index(['date', 'family'], dtype='object')
Index(['store_nbr', 'sales', 'onpromotion', 'isHoliday', 'oil_price'], dtype='object')


In [15]:
all_data['year'] = pd.to_numeric(all_data['date'].str[:4])
all_data['month'] = pd.to_numeric(all_data['date'].str[5:7])
all_data['day'] = pd.to_numeric(all_data['date'].str[8:10])

all_data['date_'] = (all_data['year']*10000) + (all_data['month']*100) + all_data['day']

In [178]:
save_point = all_data

In [179]:
all_data = save_point
len(all_data)

3029400

In [16]:
all_data_index = all_data.index

dummy_data = pd.get_dummies(data_object.drop(['date'], axis=1), drop_first=True)
dummy_data.index = all_data_index

In [17]:
dump = data_non_object
dump.index = all_data_index

In [18]:
all_data = pd.merge(dummy_data, dump, right_index=True, left_index=True)

In [20]:
nTrain = len(train)
train = all_data[:nTrain]
test = all_data[nTrain:]

In [22]:
col_x = train.columns
col_y = 'sales'

In [23]:
cormat = train.corr()

dump = cormat.index[abs(cormat['sales']) >= 0.1]
dump

Index(['family_BEVERAGES', 'family_CLEANING', 'family_GROCERY I',
       'family_PRODUCE', 'sales', 'onpromotion'],
      dtype='object')

In [32]:
from sklearn.model_selection import train_test_split

train_x = train.drop(['sales'], axis=1)
train_y = np.log1p(train['sales'])

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, shuffle=True)

test_id_idx = test.index

test_x = test.drop(['sales'], axis=1)

In [25]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

xgb = XGBRegressor(randome_state = 37)

param = {
    'max_depth':[2,3,4],
    'n_estimators':range(550,700,50),
    'colsample_bytree':[0.5,0.7,1],
    'colsample_bylevel':[0.5,0.7,1],
}
grid_search = GridSearchCV(estimator=xgb, param_grid=param, cv=2, 
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)

#print(train_x.dtypes)
#print(train_y.dtypes)

grid_search.fit(train_x, train_y)
print(grid_search.best_params_)
print(grid_search.best_estimator_)

[10:21:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
{'colsample_bylevel': 1, 'colsample_bytree': 1, 'max_depth': 4, 'n_estimators': 650}
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=None, n_estimators=650,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             randome_state=37, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)


In [26]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
pred_train = xgb.predict(train_x)
pred_val = xgb.predict(val_x)

print('train mae score: ', mean_absolute_error(train_y, pred_train))
print('val mae score:', mean_absolute_error(val_y, pred_val))

# train mae score:  0.7586943958044394
# val mae score: 0.942235584465081
# real score : 2.37214

# train mae score:  0.466127593673133
# val mae score: 0.5367417374485141
# real score : 2.47333

train mae score:  0.466127593673133
val mae score: 0.5367417374485141


In [33]:
xgb.fit(train_x, train_y)
pred = xgb.predict(test_x)

[10:26:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [27]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/sample_submission.csv')

In [34]:
#pred = grid_search.predict(test_x)
submission['sales'] = np.expm1(pred)
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/submission.csv', index=False)